# Build Recurrent Neural Networks from scratch using Tensorflow

In this notebook, we will build a vanilla recurrent neural network (RNN) from the ground up in Tensorflow, and then translate the model into Tensorflow’s RNN API.

# 1. Introduction

In this notebook, the model will be as simple as possible: at time step $t$, for $t \in \{0,1,…n\}$ the model accepts a input vector $X_t$ and a previous state vector $S_{t−1}$ as inputs and produces a state vector $S_t$, and a predicted probability distribution vector $P_t$, for the output vector $Y_t$.

Formally, the model is:
$$
S_t=\textbf{tanh}(W([X_t, S_{t−1})]+b_s)
$$
$$
P_t=\textbf{softmax}(U S_t + b_p)
$$
where $[X_t, S_{t−1}]$ represents vector concatenation, $X_t \in R^n$ is a input vector, $W \in R^{d \times (n+d)}$, $b_s \in R^d$, $U \in R^{n \times d}$, $b_p \in R^n$ and d is the size of the state vector (I use $d=4$ below). At time step 0, $S_{−1}$ (the initial state) is initialized as a vector of zeros.

Here is a diagram of the model:

![Basic RNN](.\BasicRNN.png)

# 2. “Truncate”  backpropagation through time
To build models in Tensorflow generally, you first represent the model as a graph, and then execute the graph. A critical question we must answer when deciding how to represent our model is: how wide should our graph be? How many time steps of input should our graph accept at once?

Each time step is a duplicate, so it might make sense to have our graph, $G$, represent a single time step: $G(X_t,S_{t−1}) -> (P_t,S_t)$. We can then execute our graph for each time step, feeding in the state returned from the previous execution into the current execution. This would work for a model that was already trained, but there’s a problem with using this approach for training: the gradients computed during backpropagation are graph-bound. We would only be able to backpropagate errors to the current timestep; we could not backpropagate the error to time step t-1. This means our network will not be able to learn how to store long-term dependencies (such as the two in our data) in its state.

Alternatively, we might make our graph as wide as our data sequence. This often works, except that in this case, we have an arbitrarily long input sequence, so we have to stop somewhere. Let’s say we make our graph accept sequences of length 10,000. This solves the problem of graph-bound gradients, and the errors from time step 9999 are propagated all the way back to time step 0. Unfortunately, such backpropagation is not only (often prohibitively) expensive, but also ineffective, due to the vanishing / exploding gradient problem: it turns out that backpropagating errors over too many time steps often causes them to vanish (become insignificantly small) or explode (become overwhelmingly large). 

The usual pattern for dealing with very long sequences is therefore to “truncate” our backpropagation by backpropagating errors a maximum of n steps. We choose n as a hyperparameter to our model, keeping in mind the trade-off: higher n lets us capture longer term dependencies, but is more expensive computationally and memory-wise.

Our graph will be $n$ units (time steps) wide where each unit is a perfect duplicate, sharing the same variables. The easiest way to build a graph containing these duplicate units is to build each duplicate part in parallel. This is a key point, so I’m bolding it: the easiest way to represent each type of duplicate tensor (the rnn inputs, the rnn outputs (hidden state), the predictions, and the loss) is as a list of tensors. Here is a diagram with references to the variables used in the code below:

![Basic RNN Labeled](.\BasicRNNLabeled.png)

We will run a training step after each execution of the graph, simultaneously grabbing the final state produced by that execution to pass on to the next execution.

## 3. Toy example example formulation

In this notebook, we’ll be building a raw RNN that accepts a binary sequence X and uses it to predict a binary sequence Y. The sequences are constructed as follows:

- Input sequence ($X$): At time step $t$, $X_t$ has a 50% chance of being 1 (and a 50% chance of being 0). E.g., $X$ might be $[1, 0, 0, 1, 1, 1 … ]$.

- Output sequence ($Y$): At time step $t$, $Y_t$ has a base 50% chance of being 1 (and a 50% base chance to be 0). The chance of $Y_t$ being 1 is increased by 50% (i.e., to 100%) if $X_{t−3}$ is 1, and decreased by 25% (i.e., to 25%) if $X_{t−8}$ is 1. If both $X_{t−3}$ and $X_{t−8}$ are 1, the chance of $Y_t$ being 1 is 50% + 50% - 25% = 75%.

| $X_{t-3}$| $X_{t-8}$| $p(X_{t-3}, X_{t-8})$| $p(Y_t=1 | X_{t-3}, X_{t-8})$ | correct prediction probability |
| -------- |--------| -------- |:--------:| -------- |
| 0   |  0  | 0.25 | 0.5  | 0.5  |
| 1   |  0  | 0.25 | 1.0  | 1.0  |
| 0   |  1  | 0.25 | 0.25  | 0.75  |
| 1   |  1  | 0.25 | 0.75  | 0.75  |


Thus, there are two dependencies in the data: one at $t-3$ (3 steps back) and one at $t-8$ (8 steps back).

This data is simple enough that we can calculate the expected cross-entropy loss for a trained RNN depending on whether or not it learns the dependencies:

- If the network learns no dependencies, it will correctly assign a probability of 62.5% to 1, for an expected cross-entropy loss of about 0.66. 
 - correct prediction probability = $0.25 \times (0.5 + 1.0 + 0.75 + 0.75) = 0.625$
- If the network learns only the first dependency (3 steps back) but not the second dependency, it will correctly assign a probability of 87.5%, 50% of the time, and correctly assign a probability of 62.5% the other 50% of the time, for an expected cross entropy loss of about 0.52.
 - for $p(X_{t-3} = 0) = 0.5$, correct prediction probability = $0.5 \times (0.5 + 0.75) = 0.625$
 - for $p(X_{t-3} = 1) = 0.5$, correct prediction probability = $0.5 \times (1.0 + 0.75) = 0.875$
- If the network learns both dependencies, it will be 100% accurate 25% of the time, correctly assign a probability of 50%, 25% of the time, and correctly assign a probability of 75%, 50% of the time, for an expected cross extropy loss of about 0.45.
 - for $p(X_{t-3} = 0, X_{t-8} = 0) = 0.25$, correct prediction probability = 0.5
 - for $p(X_{t-3} = 1, X_{t-8} = 0) = 0.25$, correct prediction probability = 1.0
 - for $p(X_{t-3} = 0, X_{t-8} = 1) = 0.25$, correct prediction probability = 0.75
 - for $p(X_{t-3} = 1, X_{t-8} = 1) = 0.25$, correct prediction probability = 0.75

Here are the calculations:

In [ ]:
import numpy as np

print("Expected cross entropy loss if the model:")
print("- learns neither dependency:", -(0.625 * np.log(0.625) +
                                      0.375 * np.log(0.375)))
# Learns first dependency only ==> 0.51916669970720941
print("- learns first dependency:  ",
      -0.5 * (0.875 * np.log(0.875) + 0.125 * np.log(0.125))
      -0.5 * (0.625 * np.log(0.625) + 0.375 * np.log(0.375)))
print("- learns both dependencies: ", -0.50 * (0.75 * np.log(0.75) + 0.25 * np.log(0.25))
      - 0.25 * (2 * 0.50 * np.log (0.50)) - 0.25 * (0))

# 4. Data Generation

In [ ]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Global config variables
num_steps = 5 # number of truncated backprop steps ('n' in the discussion above)
batch_size = 200
num_classes = 2
state_size = 4
learning_rate = 0.1

In [ ]:
def gen_data(size=1000000):
    X = np.array(np.random.choice(2, size=(size,)))
    Y = []
    for i in range(size):
        threshold = 0.5
        if X[i-3] == 1:
            threshold += 0.5
        if X[i-8] == 1:
            threshold -= 0.25
        if np.random.rand() > threshold:
            Y.append(0)
        else:
            Y.append(1)
    return X, np.array(Y)

In [ ]:
# adapted from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/rnn/ptb/reader.py
def gen_batch(raw_data, batch_size, num_steps):
    raw_x, raw_y = raw_data
    data_length = len(raw_x)

    # partition raw data into batches and stack them vertically in a data matrix
    batch_partition_length = data_length // batch_size
    data_x = np.zeros([batch_size, batch_partition_length], dtype=np.int32)
    data_y = np.zeros([batch_size, batch_partition_length], dtype=np.int32)
    for i in range(batch_size):
        data_x[i] = raw_x[batch_partition_length * i:batch_partition_length * (i + 1)]
        data_y[i] = raw_y[batch_partition_length * i:batch_partition_length * (i + 1)]
    # further divide batch partitions into num_steps for truncated backprop
    epoch_size = batch_partition_length // num_steps

    for i in range(epoch_size):
        x = data_x[:, i * num_steps:(i + 1) * num_steps]
        y = data_y[:, i * num_steps:(i + 1) * num_steps]
        yield (x, y)

def gen_epochs(n, num_steps):
    for i in range(n):
        yield gen_batch(gen_data(), batch_size, num_steps)

The gen_batch function splits the data into batches. There are *epoch_size* batches, and each batch is of size *batch_size*, and each training example is of length *num_steps*, as the following pictures shows:
![batch splitting](.\batch.png)

# 5. Experiments

## 5.1 placeholders and inputs

In [ ]:
"""
Placeholders
"""
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size, state_size])

In [ ]:
"""
RNN Inputs
"""
# Turn our x placeholder into a list of one-hot tensors:
# rnn_inputs is a list of num_steps tensors with shape [batch_size, num_classes]
x_one_hot = tf.one_hot(x, num_classes)
rnn_inputs = tf.unstack(x_one_hot, axis=1)

## 5.2 Definition of the RNN

### 5.2.1 Raw definition

In [ ]:
"""
Definition of rnn_cell

This is very similar to the __call__ method on Tensorflow's BasicRNNCell. See:
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py#L95
"""
with tf.variable_scope('rnn_cell'):
    W = tf.get_variable('W', [num_classes + state_size, state_size])
    b = tf.get_variable('b', [state_size], initializer=tf.constant_initializer(0.0))

def rnn_cell(rnn_input, state):
    with tf.variable_scope('rnn_cell', reuse=True):
        W = tf.get_variable('W', [num_classes + state_size, state_size])
        b = tf.get_variable('b', [state_size], initializer=tf.constant_initializer(0.0))
    return tf.tanh(tf.matmul(tf.concat([rnn_input, state], 1), W) + b)

In [ ]:
"""
Adding rnn_cells to graph

This is a simplified version of the "static_rnn" function from Tensorflow's api. See:
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/rnn/python/ops/core_rnn.py#L41
Note: In practice, using "dynamic_rnn" is a better choice that the "static_rnn":
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/rnn.py#L390
"""
state = init_state
rnn_outputs = []
for rnn_input in rnn_inputs:
    state = rnn_cell(rnn_input, state)
    rnn_outputs.append(state)
final_state = rnn_outputs[-1]

### 5.2.2 TensorFlow Definition

In [ ]:
cell = tf.contrib.rnn.BasicRNNCell(state_size)
rnn_outputs, final_state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=init_state)

## 5.3 Predictions, loss, training step

In [ ]:
"""
Predictions, loss, training step

Losses is similar to the "sequence_loss"
function from Tensorflow's API, except that here we are using a list of 2D tensors, instead of a 3D tensor. See:
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/seq2seq/python/ops/loss.py#L30
"""

#logits and predictions
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes])
    b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
logits = [tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs]
predictions = [tf.nn.softmax(logit) for logit in logits]

# Turn our y placeholder into a list of labels
y_as_list = tf.unstack(y, num=num_steps, axis=1)

#losses and train_step
losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logit) for \
          logit, label in zip(logits, y_as_list)]
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(total_loss)

## 5.4 Train the network

In [ ]:
"""
Train the network
"""

def train_network(num_epochs, num_steps, state_size=4, verbose=True):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        training_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, num_steps)):
            training_loss = 0
            training_state = np.zeros((batch_size, state_size))
            if verbose:
                print("\nEPOCH", idx)
            for step, (X, Y) in enumerate(epoch):
                tr_losses, training_loss_, training_state, _ = \
                    sess.run([losses,
                              total_loss,
                              final_state,
                              train_step],
                                  feed_dict={x:X, y:Y, init_state:training_state})
                training_loss += training_loss_
                if step % 100 == 0 and step > 0:
                    if verbose:
                        print("Average loss at step", step,
                              "for last 250 steps:", training_loss/100)
                    training_losses.append(training_loss/100)
                    training_loss = 0

    return training_losses

In [ ]:
training_losses = train_network(1,num_steps)
plt.plot(training_losses)

## 5.5 Final model — Using a dynamic RNN

Above, we added every node for every timestep to the graph before execution. This is called “static” construction. We could also let Tensorflow dynamically create the graph at execution time, which can be more efficient. To do this, instead of using a list of tensors (of length num_steps and shape [batch_size, features]), we keep everything in a single 3-dimnesional tensor of shape [batch_size, num_steps, features], and use Tensorflow’s dynamic_rnn function. This is shown below.

In [ ]:
"""
Placeholders
"""
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size, state_size])

"""
Inputs
"""
rnn_inputs = tf.one_hot(x, num_classes)

"""
RNN
"""
cell = tf.contrib.rnn.BasicRNNCell(state_size)
rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)

"""
Predictions, loss, training step
"""
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes])
    b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
logits = tf.reshape(
            tf.matmul(tf.reshape(rnn_outputs, [-1, state_size]), W) + b,
            [batch_size, num_steps, num_classes])
predictions = tf.nn.softmax(logits)

losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(total_loss)

## 5.6 TODO 

Try different combination of the hyper-parameters, and retrain to see change of the loss. Please try: 
- num_steps = 1, 5, 10
- state_size = 4, 16

# 6. References

- [1] [Understanding LSTM Networks, by Christopher Olah](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [2] [The Unreasonable Effectiveness of Recurrent Neural Networks, by Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [3] [Recurrent Neural Networks Tutorial, by Denny Britz](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)
- [4] [Recurrent Neural Networks in Tensorflow Part I, from R2RT](http://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html)
- [5] [Creating A Text Generator Using Recurrent Neural Network, by Trung Tran](https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/)